<a href="https://colab.research.google.com/github/basharE/hecht/blob/master/DeepLearning_Ex3_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Training Model for Object Detection Task


## Preparing Environment for Training 


###Background
Colab has two versions of TensorFlow pre-installed: a 2.x version and a 1.x version. Colab uses TensorFlow 2.x by default, though you can switch to 1.x by the method shown below.


###Specifying the TensorFlow version

Running `import tensorflow` will import the default version (currently 2.x). You can use 1.x by running a cell with the `tensorflow_version` magic **before** you run `import tensorflow`.

In [ ]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


### Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Import Tensorflow
Note: as described above the tf version will imported is 1.x.

In [ ]:
import tensorflow
print(tensorflow.__version__)

1.15.2


### Navigate to Object Detection Directory

In [ ]:
%cd /content/drive/MyDrive/Anaconda/models/research/object_detection

/content/drive/MyDrive/Anaconda/models/research/object_detection


### Enabling and testing the GPU

First, you'll need to enable GPUs for the notebook:

- Navigate to Edit→Notebook Settings
- select GPU from the Hardware Accelerator drop-down

Next, we'll confirm that we can connect to the GPU with tensorflow:

In [ ]:
%tensorflow_version 1.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


## Preparing Data

### Create CSV files from XML

In [ ]:
!python xml_to_csv.py

Successfully converted xml to csv.
Successfully converted xml to csv.


### Generate TF Records from created CSV

In [ ]:
!python generate_tfrecord.py --csv_input=images_ahmad/train_labels.csv --image_dir=images_ahmad/train --output_path=train.record
!python generate_tfrecord.py --csv_input=images_ahmad/test_labels.csv --image_dir=images_ahmad/test --output_path=test.record

Successfully created the TFRecords: /content/drive/My Drive/Anaconda/models/research/object_detection/train.record
Successfully created the TFRecords: /content/drive/My Drive/Anaconda/models/research/object_detection/test.record


In [ ]:
import os
os.environ['PYTHONPATH'] += ':/content/drive/MyDrive/Anaconda/models/research:/content/drive/MyDrive/Anaconda/models/research/slim'

### Install Missing Libraries

In [ ]:
!pip install tf_slim
!pip install lvis

     |████████████████████████████████| 358kB 14.2MB/s 


## Train Model

### Run Model Training

In [ ]:
!python model_main.py --logtostderr --model_dir=train_dir/ --pipeline_config_path=train_dir/faster_rcnn_inception_v2_pets.config

W0205 08:55:25.265760 140180329301888 model_lib.py:793] Forced number of epochs for all eval validations to be 1.
INFO:tensorflow:Maybe overwriting train_steps: None
I0205 08:55:25.266032 140180329301888 config_util.py:552] Maybe overwriting train_steps: None
INFO:tensorflow:Maybe overwriting use_bfloat16: False
I0205 08:55:25.266128 140180329301888 config_util.py:552] Maybe overwriting use_bfloat16: False
INFO:tensorflow:Maybe overwriting sample_1_of_n_eval_examples: 1
I0205 08:55:25.266218 140180329301888 config_util.py:552] Maybe overwriting sample_1_of_n_eval_examples: 1
INFO:tensorflow:Maybe overwriting eval_num_epochs: 1
I0205 08:55:25.266303 140180329301888 config_util.py:552] Maybe overwriting eval_num_epochs: 1
W0205 08:55:25.266424 140180329301888 model_lib.py:809] Expected number of evaluation epochs is 1, but instead encountered `eval_on_train_input_config.num_epochs` = 0. Overwriting `num_epochs` to 1.
INFO:tensorflow:create_estimator_and_inputs: use_tpu False, export_to_t

In [ ]:
%cd /content/drive/MyDrive/Anaconda/models/research/object_detection

/content/drive/MyDrive/Anaconda/models/research/object_detection


###Insatlling Object Detection API

In [ ]:
!pip install tensorflow-object-detection-api

     |████████████████████████████████| 583kB 14.7MB/s 
     |████████████████████████████████| 61kB 9.8MB/s 
     |████████████████████████████████| 51kB 6.4MB/s 
     |████████████████████████████████| 2.6MB 27.9MB/s 
  Created wheel for tensorflow-object-detection-api: filename=tensorflow_object_detection_api-0.1.1-cp36-none-any.whl size=844516 sha256=5f17cc4f6092979bf9f13a32e7673d4fab9023b2ce069f025c324b96be3217dd
  Stored in directory: /root/.cache/pip/wheels/4a/54/d0/cfca11930c4b2025d40dede77059094070a67cc3e7bd3b285f
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=6577f2c18dde019271c3279332e17969c583da1b43b71f3dd360eb8ca661b0c8
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built tensorflow-object-detection-api gast
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3


### Export Inference Graph

In [ ]:
!python export_inference_graph.py \
    --input_type image_tensor \
    --pipeline_config_path train_dir/faster_rcnn_inception_v2_pets.config \
    --trained_checkpoint_prefix train_dir/model.ckpt-200000 \
    --output_directory inference_graph_ahmad

Instructions for updating:
Please use `layer.__call__` method instead.
W0205 08:55:58.194593 139688781322112 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tf_slim/layers/layers.py:2802: Layer.apply (from tensorflow.python.keras.engine.base_layer) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `layer.__call__` method instead.
INFO:tensorflow:Scale of 0 disables regularizer.
I0205 08:55:59.388578 139688781322112 regularizers.py:99] Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
I0205 08:55:59.537700 139688781322112 regularizers.py:99] Scale of 0 disables regularizer.
INFO:tensorflow:depth of additional conv before box predictor: 0
I0205 08:55:59.538085 139688781322112 convolutional_box_predictor.py:156] depth of additional conv before box predictor: 0
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0205 08:55:59.592986 139688781322112 deprecati